<a href="https://colab.research.google.com/github/davidnoone/GEOPHYS_NOTEBOOKS/blob/main/ClimateVariability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Climate variability

In [ ]:
# Obtain the data files


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import linalg
import matplotlib
import matplotlib.pyplot as plt
try:
  import netCDF4 as nc
except:
  !pip install netCDF4
  import netCDF4 as nc


In [ ]:
# open and read the netcdf files

In [ ]:
# open and read the MEI data file
file_mei = 'meiv2.data'
columns = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 
                   'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
df = pd.read_csv(file_mei, sep='\s+', header=None, names=columns)


Example function to perform PCA

In [ ]:
def pca(X):
  """ A simple principle component analysis """
  
  # Compute the mean, and center the data 
  mean = np.mean(X, axis=0)
  X_centered = X - mean

  # Compute the covariance matrix
  covariance_matrix = np.cov(X_centered, rowvar=False)

  # Calulate the eigen values/vectors of the covariance
  eigenvalues, eigenvectors = linalg.eigh(covariance_matrix)

  # Sort these, since python (actually LAPACK!) doesn't do it
  idx = np.argsort(eigenvalues)[::-1]
  eigenvalues = eigenvalues[idx]
  eigenvectors = eigenvectors[:, idx]

  ## Select just the top "numcomponents"
  #X_transformed = np.dot(X_centered, eigenvectors[:, :num_components])

  return eigenvalues, eigenvectors

In [ ]:
nlon = 144
nlat = 72
fake_data = np.random.random((nlon,nlat))

eval, evec = pca(fake_data.flatten())



##Part 1: regression maps

##Part 2: patterns of variability